In [23]:
source("tools.R")

Parsed with column specification:
cols(
  .default = col_double(),
  userId = col_character(),
  age = col_character(),
  datetime = col_character(),
  gender = col_character(),
  datetime_1 = col_character(),
  stage = col_integer(),
  scenario = col_character(),
  subcondition = col_integer(),
  pageIndex = col_integer(),
  noiseIndex = col_integer(),
  day0 = col_character(),
  day1 = col_character(),
  day2 = col_character(),
  day3 = col_character(),
  day4 = col_character(),
  day5 = col_character(),
  day6 = col_character(),
  day7 = col_character(),
  day8 = col_character(),
  day9 = col_character()
  # ... with 69 more columns
)
See spec(...) for full column specifications.
Warning message in eval(substitute(expr), envir, enclos):
"NAs introduced by coercion"

In [24]:
library(gridExtra)
library(lsr)

##### Bootstrapped confidence interval (for binomial distribution).

NOTE: Do you like this method Eric? Or should we use a package [like this one](http://math.furman.edu/~dcs/courses/math47/R/library/Hmisc/html/binconf.html)?

In [25]:
bp = function(x, lev, n = 1e4, alpha=0.05) {
  res = replicate(n, sum(sample(x, length(x), replace=TRUE) == lev)/length(x))
  return(list(mean=mean(res),
              `95% CI`=quantile(res, c(0.5*alpha,1-0.5*alpha))))
}

bp2 = function(vector, n = 1e4, alpha=0.05, round_n=2) {
    
    match_ci <- bp(x=vector, lev=TRUE, n=n, alpha=alpha)
    
    return( c( match_ci$mean %>% round(round_n),
             match_ci$`95% CI`[1] %>% round(round_n),
             match_ci$`95% CI`[2] %>% round(round_n) ))
}

In [26]:
standardize_range_one <- function(array) {
    (array - min(array)) / (max(array) - min(array))
}

In [27]:
# Kernel composition name change
#l+r+p -> l+p+r
#l+r*p -> l+p*r
#l*r*p -> l*p*r

readable_kernel <- function(current_kernel){
    new_kernel <- current_kernel
    
    new_kernel <- ifelse(new_kernel == 'l+r+p', 'l+p+r', new_kernel)
    new_kernel <- ifelse(new_kernel == 'l+r*p', 'l+p*r', new_kernel)
    new_kernel <- ifelse(new_kernel == 'l*r*p', 'l*p*r', new_kernel)
    
    new_kernel <- gsub("\\*", "×", new_kernel)
    
    return(new_kernel)
}

# Get the range of a scenario
get_range <- function(scenario_name) {
    switch( scenario_name,
            'Temperature' = c(-10, 40),
            'Rain' = c(0, 100),
            'Sales' = c(0, 5000),
            'Gym members' = c(0, 50),
            'Salary' = c(0, 50),
            'FB Friends' = c(0, 1000))
}

kernels  <- c("l", "p", "r", "l+p", "l+r", "p+r", "l*r", "l*p", "p*r", "l+r+p", "l+r*p", "l*r+p", "l*p+r", "l*r*p")

# Plotting functions

In [28]:
plot_lmls <- function(prop_data, title='', scenario, hide_x=FALSE, hide_y=FALSE, red_border=TRUE) {
    
    # Best-fitting real-world data kernel composition
    rw_lmls <- read_csv("data/real-world/to-plot.csv")
    red_kernel <- (rw_lmls %>%
                    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
                    filter(scenario == !!scenario) %>%
                    summarize(red_kernel = kernel[which.max(lml)]))$red_kernel
    red_kernel <- readable_kernel(red_kernel)

    plot <- prop_data %>%
                ggplot(aes(x=kernel, y=value)) +
                    geom_bar(stat='identity') +
                    geom_bar(stat='identity', data=prop_data %>% filter(kernel == red_kernel), alpha=0, size=0.5, color="red") +
                    coord_cartesian (ylim=c(0, 1)) +
                    #labs(title = title) +
                    ggthemes::theme_few() +
                    xlab("Kernel composition") +
                    ylab("Likelihood") +
                    geom_errorbar(aes(ymin=as.numeric(lo_ci), ymax=as.numeric(hi_ci)),
                                      width=.3,
                                      position=position_dodge(.9)) +
                    scale_y_continuous(breaks = seq(0, 1, length.out=3)) +
                    theme(axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                          text = element_text(size=12, family="serif"))

    if(hide_x){
        plot <- plot + theme(axis.title.x=element_blank())
    }
    if(hide_y){
        plot <- plot + theme(axis.title.y=element_blank())
    }
    
    return(plot)
}

In [29]:
plot_curves <- function(plot_data, plot_rwdata, ylab='', scenario, hide_x=FALSE, hide_y=FALSE) {

    # Min and max value of the data to be shown
    range_y <- get_range(scenario)
    limits_y <- c( min(range_y[1], min(plot_data$value)), max(range_y[2], max(plot_data$value)))
    
    # Mean trend
    trend <- plot_data %>% group_by(day) %>% summarize(mean_y = mean(value))
    
    # Resolution subsetting
    resolution = 10 #smaller number is higher resolution
    plot_data <- plot_data %>% filter((day - min(day)) %% resolution == 0)
    trend <- trend %>% filter((day - min(day)) %% resolution == 0)
    
    # Plotting
    plot <- plot_data %>%
                ggplot(aes(x=day, y=value, group=id)) +
                    geom_line(col="steelblue", alpha=0.2) + # Participants' curves
                    geom_line(data=plot_rwdata, aes(x=day, group=1), colour="red", alpha=0.6) + # Real world data
                    geom_line(data=trend, aes(x=day, y=mean_y, group=1), colour="black") + # Mean trend
                    #labs(title = title) +
                    ggthemes::theme_few() +
                    ylab(ylab) +
                    scale_x_continuous(breaks = c(0, 365, 365*2, 365*3), labels=c('Y1', 'Y2', 'Y3', 'Y4')) +
                    scale_y_continuous(breaks = seq(range_y[1], range_y[2], length.out=3), limits=limits_y) +
                    theme(axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                          axis.text.y = element_text(angle = 90,  vjust = 0.5, hjust=0.5),
                          text = element_text(size=12, family="serif"))

    if(hide_x){
        plot <- plot + theme(axis.title.x=element_blank())
    }
    if(hide_y){
        plot <- plot + theme(axis.title.y=element_blank())
    }
    
    return(plot)
}

# Prior: Kernel compositions bar plots

##### Import and prepare the data

In [30]:
data_prior <- read_csv("data/for_composititional_analysis_prior.csv")

# Get a "dictionary" of ID, PID, Scenario
dict_prior <- data_prior %>%                        
                        group_by(id, pid, scenario) %>%
                        summarize()

# Get the results of the LML analysis over the prior data
prior_results <- read_csv('output/minus-mean-treatment/results_prior_lmls.csv')

# Merge the results with the dictionary to have a 'full view' of the situation
lmls_prior <- merge(prior_results, dict_prior, by.x='cid', by.y='id', all.x=TRUE)

# Kernel composition name change
lmls_prior$kernel <- readable_kernel(lmls_prior$kernel)

Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_integer(),
  id = col_integer(),
  pid = col_character(),
  scenario = col_character(),
  x = col_integer(),
  y = col_double()
)
Parsed with column specification:
cols(
  cid = col_integer(),
  kernel = col_character(),
  lml = col_double(),
  white_added = col_character(),
  second_exception = col_character()
)


In [31]:
# Removes the GPs that failed to be optimized in the second time. The lmls that failed to be optimized are also removed.
#Finaly, the lml_standard is calculated
lmls_prior_f <- lmls_prior %>%
                    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
                    filter(second_exception == 'False' & lml != -999999999)

sds <- lmls_prior_f %>% 
            group_by(kernel, scenario) %>%
            summarize(lo_bound = mean(lml) - 5*sd(lml), up_bound = mean(lml) + 5*sd(lml))

sds$ks <- paste0(sds$kernel, sds$scenario)

lmls_prior_f$ks <- paste0(lmls_prior_f$kernel, lmls_prior_f$scenario)

sds$kernel <- NULL
sds$scenario <- NULL

lmls_prior_f <- merge(x = lmls_prior_f, y = sds, by = "ks", all.x = TRUE)

##### Prepare the plots

In [32]:
plot_bool      = TRUE

lmls_prior_ff <- lmls_prior_f %>%
            filter(lml > lo_bound & lml < up_bound)

# Remove single components AND for each participant-scenario combination, make a 0 to 1 range
to_plot_prior <- lmls_prior_ff %>%
                    group_by( pid, scenario ) %>%
                    mutate( lml = standardize_range_one(lml) )

# Produce means and confidence intervals
to_plot_prior <- to_plot_prior %>%
            group_by(kernel, scenario) %>%
            summarize(value = mean(lml),
                      lo_ci = MeanCI(lml, method="boot", type="norm", na.rm=TRUE)['lwr.ci'],
                      hi_ci = MeanCI(lml, method="boot", type="norm", na.rm=TRUE)['upr.ci'])

# Plotting magic
to_plot_prior$kernel <- factor(to_plot_prior$kernel, levels=readable_kernel(kernels))


if (plot_bool) {
    # Plots
    p_temp   <- plot_lmls( to_plot_prior %>% filter(scenario == "Temperature"), scenario="Temperature", hide_x=TRUE, hide_y=FALSE)
    p_rain   <- plot_lmls( to_plot_prior %>% filter(scenario == "Rain"),        scenario="Rain",        hide_x=TRUE, hide_y=FALSE)
    p_sales  <- plot_lmls( to_plot_prior %>% filter(scenario == "Sales"),       scenario="Sales",       hide_x=TRUE, hide_y=FALSE)
    p_gym    <- plot_lmls( to_plot_prior %>% filter(scenario == "Gym members"), scenario="Gym members", hide_x=TRUE, hide_y=FALSE)
    p_salary <- plot_lmls( to_plot_prior %>% filter(scenario == "Salary"),      scenario="Salary",      hide_x=TRUE, hide_y=FALSE)
    p_fb     <- plot_lmls( to_plot_prior %>% filter(scenario == "FB Friends"),  scenario="FB Friends",  hide_x=TRUE, hide_y=FALSE)

    #pdf("Images/paper_images/kernels_priors_lmls_5sd.pdf", width=8, height=4)
    #multiplot(p1, p2, p3, p4, p5, p6, cols=3)
    #dev.off()   
}

Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)


## Tests for Prior condition

#### Best fitting kernel compositions

In [33]:
to_plot_prior %>%
    group_by(scenario) %>%
    summarize(composition = kernel[which.max(value)], value = value[which.max(value)]) %>%
    arrange(scenarios_order(scenario))

# Which other kernel compositions are statistically equivalent
to_plot_prior %>%
    group_by(scenario) %>%
    filter(hi_ci > lo_ci[which.max(value)]) %>%
    select(scenario, composition = kernel, value, lo_ci, hi_ci) %>%
    arrange(scenarios_order(scenario), -value)

scenario,composition,value
Temperature,p×r,0.9192010
Rain,p×r,0.9185013
Sales,l+r,0.9652802
Gym members,p×r,0.8959303
Salary,l+r,0.8576757
FB Friends,l+r,0.9315774


scenario,composition,value,lo_ci,hi_ci
Temperature,p×r,0.9192010,0.8793220,0.9586133
Rain,p×r,0.9185013,0.8821189,0.9536117
Rain,l+r,0.8727970,0.8462698,0.8989480
Rain,l+p×r,0.8647992,0.8246495,0.9037312
Sales,l+r,0.9652802,0.9508533,0.9804097
Gym members,p×r,0.8959303,0.8581429,0.9349186
Salary,l+r,0.8576757,0.8172323,0.8979050
Salary,p+r,0.8078211,0.7498714,0.8645543
FB Friends,l+r,0.9315774,0.9156421,0.9460094
FB Friends,l+p×r,0.8974742,0.8690597,0.9273166


#### Percentage of removed GPs in the Prior condition:

In [34]:
# Percentage of removed GPs in the Prior condition:
((1 -
lmls_prior_ff %>%
    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
    nrow / 
lmls_prior %>%
    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
    nrow) * 100) %>%
round(2)


[1] 5.02

#### Ranking correlation: participants -> real-world

In [35]:
# Get the real world data and make it readable
to_merge_rw_lmls <- read_csv("data/real-world/to-plot.csv")
to_merge_rw_lmls$kernel <- readable_kernel(to_merge_rw_lmls$kernel)

# Filter out the non-compositional kernels
to_merge_rw_lmls <- to_merge_rw_lmls %>% 
                filter(kernel != 'l', kernel != 'p', kernel != 'r')

# Raw prior data: lmls_prior 
to_merge_lmls_prior <- lmls_prior_ff %>% filter(kernel != 'l', kernel != 'p', kernel != 'r')

# Merging the real world data with the prior results
lmls_prior_new <- merge(x = to_merge_lmls_prior, 
                        y = to_merge_rw_lmls,
                        by = c('scenario', 'kernel'),
                        suffixes = c("_participants","_real_world")
                        )

Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)


In [36]:
# Calculate the rank correlation
print('Per scenario')

dcor1 <- ddply(lmls_prior_new, ~scenario+pid, summarize, r=cor(rank(lml_participants), rank(lml_real_world)))

dout <- ddply(dcor1, ~scenario, summarize, m=cohensD(r))
test_results_per_scenario <- tapply(dcor1$r, dcor1$scenario, t.test)
print('Overall')

dcor2 <- ddply(lmls_prior_new, ~pid, summarize, r=cor(rank(lml_participants), rank(lml_real_world)))

t.test(dcor2$r)
cohensD(dcor2$r) %>% round(2)

[1] "Per scenario"
[1] "Overall"



	One Sample t-test

data:  dcor2$r
t = 13.912, df = 118, p-value < 2.2e-16
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.1184958 0.1578284
sample estimates:
mean of x 
0.1381621 


[1] 1.28

In [37]:
print_test_results <- function(scenario_name, readable_s_name){
    df <- test_results_per_scenario[[scenario_name]][[2]]
    p_val <- ifelse(test_results_per_scenario[[scenario_name]][[3]] < 0.001, 
                "<0.001", 
                test_results_per_scenario[[scenario_name]][[3]])
    t <- test_results_per_scenario[[scenario_name]][[1]] %>% round(2)

    d_val <- (dout %>% filter(scenario == scenario_name))$m %>% round(2)

    rho <- test_results_per_scenario[[scenario_name]][[5]] %>% round(2)
    
    t_string <- paste0(", $\rho=", rho,"$", " ($t(", df, ")=", t, ", p", p_val, ", d=", d_val, "$)")
    
    paste0(t_string, " for the ", readable_s_name, " data")
}

In [38]:
paste0(  print_test_results('Temperature', 'Temperature'),
        print_test_results('Rain', 'Rain'),
        print_test_results('Sales', 'Sales'),
        print_test_results('Gym members', 'Gym members'),
        print_test_results('Salary', 'Salary'),
        print_test_results('FB Friends', 'Facebook friends')) %>% print

[1] ", $\rho=0.61$ ($t(118)=32.26, p<0.001, d=2.96$) for the Temperature data, $\rho=0.58$ ($t(118)=24.63, p<0.001, d=2.26$) for the Rain data, $\rho=0.48$ ($t(118)=20.23, p<0.001, d=1.85$) for the Sales data, $\rho=0.16$ ($t(118)=6.41, p<0.001, d=0.59$) for the Gym members data, $\rho=0.61$ ($t(118)=37.3, p<0.001, d=3.42$) for the Salary data, $\rho=0.62$ ($t(118)=33.62, p<0.001, d=3.08$) for the Facebook friends data"


# Prior: Curves (participant's data)

##### Plotting

In [39]:
mean_centering <- function(rwdata, dat, scenario) {
    mean_rw <- rwdata %>% filter(scenario == !!scenario) %>% summarize(mv=mean(value))
    
    mean_dat <- dat %>% filter(scenario == !!scenario, condition == 'Prior') %>% summarize(mv=mean(value))    
    
    mean_dat$mv / mean_rw$mv
}

# Real world data
rwdata <- read_csv('data/real-world/splines.csv')

rwdata$condition <- 'Prior' # Add column
rwdata$id <- 1 #Create a new ID column
rwdata <- filter(rwdata, day >= 31) # Filter out the initial days
rwdata <- filter(rwdata, day <= 365*4 - 31) # Filter out the final days

# Order
rwdata$condition <- factor( rwdata$condition, levels = condition_names)
rwdata$scenario <- factor( rwdata$scenario, levels = readable_scenarios)

# Scale the values
#rwdata[rwdata$scenario == 'Rain', 'day'] <- rwdata[rwdata$scenario == 'Rain', 'day'] + (365/12) * 1.5 #Offset 1.5 months to the right

# Mean centering
for (s in readable_scenarios){
    rwdata[rwdata$scenario == s, 'value'] <- rwdata[rwdata$scenario == s, 'value'] * mean_centering(rwdata, dat, s)
}

Parsed with column specification:
cols(
  scenario = col_character(),
  day = col_integer(),
  value = col_double()
)


In [40]:
curves_plot <- function(scenario_name, ...) {
    dat %>% 
        filter(scenario==scenario_name, condition=='Prior') %>% 
        plot_curves(rwdata %>% filter(scenario==scenario_name), scenario=scenario_name, ylab=scenario_name, ...)
}

In [41]:
p_temp_curves   <- curves_plot('Temperature', hide_x=TRUE, hide_y=FALSE)
p_rain_curves   <- curves_plot('Rain',        hide_x=TRUE, hide_y=FALSE)
p_sales_curves  <- curves_plot('Sales',       hide_x=TRUE, hide_y=FALSE)
p_gym_curves    <- curves_plot('Gym members', hide_x=TRUE, hide_y=FALSE)
p_salary_curves <- curves_plot('Salary',      hide_x=TRUE, hide_y=FALSE)
p_fb_curves     <- curves_plot('FB Friends',  hide_x=TRUE, hide_y=FALSE)

In [42]:
if(TRUE){
    # Save/show
    pdf("Images/paper_images/grid_plot_prior_new.pdf", width=6, height=10)
    grid.arrange(p_temp_curves,   p_temp, 
                 p_rain_curves,   p_rain, 
                 p_sales_curves,  p_sales, 
                 p_gym_curves,    p_gym, 
                 p_salary_curves, p_salary, 
                 p_fb_curves,     p_fb, 
                 ncol=2)
    dev.off()   
}

png 
  2

# Posterior: Curves (participants' data)

In [43]:
plot_curves_posterior <- function(plot_data, ylab='', scenario, hide_x=FALSE, hide_y=FALSE) {

    # Min and max value of the data to be shown
    range_y <- get_range(scenario)
    limits_y <- c( min(range_y[1], min(plot_data$value)), max(range_y[2], max(plot_data$value)))
    
    # Mean trend
    trend_1 <- plot_data %>% filter(condition=='Posterior-Positive') %>% group_by(day) %>% summarize(mean_y = mean(value))
    trend_2 <- plot_data %>% filter(condition=='Posterior-Stable') %>%   group_by(day) %>% summarize(mean_y = mean(value))
    trend_3 <- plot_data %>% filter(condition=='Posterior-Negative') %>% group_by(day) %>% summarize(mean_y = mean(value))
    
    # Resolution subsetting
    resolution = 10 #smaller number is higher resolution
    plot_data <- plot_data %>% filter((day - min(day)) %% resolution == 0)
    trend_1 <- trend_1 %>% filter((day - min(day)) %% resolution == 0)
    trend_2 <- trend_2 %>% filter((day - min(day)) %% resolution == 0)
    trend_3 <- trend_3 %>% filter((day - min(day)) %% resolution == 0)
    
    # Plotting
    plot <- plot_data %>%
                ggplot(aes(x=day, y=value, group=id)) +
                    # Curves:
                    geom_line(data=plot_data %>% filter(condition=='Posterior-Positive'), col="steelblue", alpha=0.1) +
                    geom_line(data=plot_data %>% filter(condition=='Posterior-Stable'),   col="olivedrab", alpha=0.1) +
                    geom_line(data=plot_data %>% filter(condition=='Posterior-Negative'), col="firebrick", alpha=0.1) +
                    
                    # Mean curves
                    geom_line(data=trend_1, aes(x=day, y=mean_y, group=1), colour="blue") +
                    geom_line(data=trend_2, aes(x=day, y=mean_y, group=1), colour="darkgreen") +
                    geom_line(data=trend_3, aes(x=day, y=mean_y, group=1), colour="red") +
                    
                    # Vertical line (end of evidence mark)
                    geom_vline(aes(xintercept=365-31), colour='black') +
    
                    ggthemes::theme_few() +
                    ylab(ylab) +
                    scale_x_continuous(breaks = c(0, 365, 365*2, 365*3), labels=c('Y1', 'Y2', 'Y3', 'Y4')) +
                    scale_y_continuous(breaks = seq(range_y[1], range_y[2], length.out=3), limits=limits_y) +
                    theme(axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                          axis.text.y = element_text(angle = 90,  vjust = 0.5, hjust=0.5),
                          text = element_text(size=12, family="serif"))

    if(hide_x){
        plot <- plot + theme(axis.title.x=element_blank())
    }
    if(hide_y){
        plot <- plot + theme(axis.title.y=element_blank())
    }
    
    return(plot)
}
    
curves_plot_posterior <- function(scenario_name, ...) {
    dat %>% 
        filter(scenario==scenario_name, condition!='Prior') %>% 
        plot_curves_posterior(scenario=scenario_name, ylab=scenario_name, ...)
}

In [44]:
posterior_curves_temp <- curves_plot_posterior('Temperature', hide_x=TRUE, hide_y=FALSE)
posterior_curves_rain <- curves_plot_posterior('Rain',        hide_x=TRUE, hide_y=FALSE)
posterior_curves_sale <- curves_plot_posterior('Sales',       hide_x=TRUE, hide_y=FALSE)
posterior_curves_gymm <- curves_plot_posterior('Gym members', hide_x=TRUE, hide_y=FALSE)
posterior_curves_slry <- curves_plot_posterior('Salary',      hide_x=TRUE, hide_y=FALSE)
posterior_curves_fbfr <- curves_plot_posterior('FB Friends',  hide_x=TRUE, hide_y=FALSE)

# Posterior: Performance. Prediction against participant's results

##### Importing the posterior kernel composition data

In [45]:
#Returns the data (after double filtering)
get_filtered_data <- function(src_lmls, src_dict) {
    lmls_posterior <- read_csv(src_lmls)

    
    # To add the 'scenario' column
    data_posterior <- read_csv(src_dict)


    # Posterior "dictionary"
    dict_posterior <- data_posterior %>%                        
                            group_by(id, pid, scenario, condition) %>%
                            summarize()

    lmls_posterior <- merge(x = lmls_posterior, y = dict_posterior, by = c("id", "id"), all.x = TRUE)

    # Removing unusable data
    lmls_posterior_f <- lmls_posterior %>%
                        filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
                        filter(second_exception == 'False' & lml != -999999999)

    # Standard deviations calculation
    sds <- lmls_posterior_f %>% 
                group_by(kernel, scenario) %>%
                summarize(lo_bound = mean(lml) - 5*sd(lml), up_bound = mean(lml) + 5*sd(lml))

    sds$ks <- paste0(sds$kernel, sds$scenario)

    lmls_posterior_f$ks <- paste0(lmls_posterior_f$kernel, lmls_posterior_f$scenario)

    sds$kernel <- NULL
    sds$scenario <- NULL

    lmls_posterior_f <- merge(x = lmls_posterior_f, y = sds, by = "ks", all.x = TRUE)

    lmls_posterior_ff <- lmls_posterior_f %>%
                filter(lml > lo_bound & lml < up_bound)
    
    return (lmls_posterior_ff)
}

In [46]:
lmls_posterior_ff <- get_filtered_data( "output/minus-mean-treatment/results_posterior_lmls.csv",
                                        "data/for_composititional_analysis_posterior.csv" )

Parsed with column specification:
cols(
  id = col_integer(),
  kernel = col_character(),
  lml = col_double(),
  white_added = col_character(),
  second_exception = col_character()
)
Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_integer(),
  id = col_integer(),
  pid = col_character(),
  scenario = col_character(),
  x = col_integer(),
  y = col_double(),
  condition = col_character()
)


###### Importing the predictions

In [47]:
# Import the predictions
predictions_csv <- read_csv('output/minus-mean-treatment/results_posterior_predictions.csv')

predictions <- predictions_csv %>%
                    mutate(value = as.numeric(substring(predictions_mean, 2, nchar(predictions_mean)-1))) %>%
                    select(cid = pid, kernel = composition, day = Xpredictions, value)

Parsed with column specification:
cols(
  pid = col_integer(),
  composition = col_character(),
  Xpredictions = col_double(),
  predictions_mean = col_character(),
  predictions_var = col_character()
)


In [48]:
# Add the subtracted mean of the 'evidence' data
posterior_means <- read_csv("data/for_composititional_analysis_posterior.csv") %>% #data_posterior
                        filter(x <= 365-31) %>%
                        group_by(id) %>%
                        summarize(mean_cid = mean(y))

predictions <- merge(x=predictions, y=posterior_means, by.x='cid', by.y='id', all.x=TRUE)

predictions <- predictions %>% 
                    mutate(value = value + mean_cid) %>%
                    select(-mean_cid)

Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_integer(),
  id = col_integer(),
  pid = col_character(),
  scenario = col_character(),
  x = col_integer(),
  y = col_double(),
  condition = col_character()
)


In [49]:
predictions %>% select(cid) %>% unique %>% nrow

predictions %>% select(cid, kernel) %>% unique %>% nrow

[1] 714

[1] 9996

##### Merging and filtering the datasets

In [50]:
# Sequence of days to analyze:
sequence = seq(365-31 + 2, 1426, 5)

In [190]:
# Inner join. 
# This is the predictions data merged with the best performing (and filtered) kernels in the posterior condition
pred <- merge( 
            # Predictions (full-Bayesian):
            x = predictions %>%
                    filter(day %in% sequence),

            # Kernels after the filters:
            y = lmls_posterior_ff %>%
                    select(cid=id, pid, scenario, kernel, lml),
            
            all.y = TRUE,
            by = c('cid', 'kernel')
)

# Participants' data
participants_data <- dat %>% 
                        filter(day > 365-31, #remove evidence
                               condition != 'Prior', #only Posterior conditions
                               day %in% sequence) %>%
                        select(pid = id, day, value_participant=value, scenario, condition, noise)

# Merge the predictions with the real participants' data
all_pred_data <- merge(
                # Predictions + Kernel posterior analysis:
                x = pred,
    
                # Participant's data:
                y = participants_data,
    
                by = c('pid', 'scenario', 'day')
)

In [189]:
all_pred_data %>% select(cid) %>% unique %>% nrow

all_pred_data %>% select(cid, kernel) %>% unique %>% nrow

[1] 714

[1] 7724

###### Calculating the deviation

In [53]:
#Normalized root mean squared deviation
calculate_nrmsd <- function(value_prediction, value_participant) {
    return (sqrt(sum((value_prediction - value_participant)^2) / length(value_prediction)) / (max(value_participant) - min(value_participant)))
}

#Normalized root mean squared error (as per Mozer et al. (2008))
calculate_nrmse <- function(value_model, value_human) {
    return ( sqrt( sum((value_human - value_model)^2) / sum((value_human - mean(value_human))^2) ) )
}

calculate_mean_error <- function(value_model, value_human) {
    return ( mean(abs(value_model - value_human)) )
}

In [54]:
add_is_max_column <- function(results_pred) {
    to_return <- merge(x = results_pred,

                      # Mark the best-fitting kernel in each case
                      y = results_pred %>%
                            filter(nchar(kernel)!= 1) %>%
                            group_by(pid, scenario) %>%
                            mutate(is_max = lml == max(lml)) %>%
                            filter(is_max) %>%
                            select(pid, scenario, kernel, is_max),

                      all.x = TRUE
                )
    
    to_return$is_max <- !(is.na(to_return$is_max))
    
    return(to_return)
}

In [55]:
results_pred <- all_pred_data %>% 
                    #filter(day <= 365*2) %>% # To Eric: this is something I though about. What do you think about it?
                    group_by(pid, scenario, kernel, lml) %>%
                    summarize(error = calculate_nrmse(value, value_participant))
                    #summarize(error = calculate_mean_error(value, value_participant))

results_pred <- add_is_max_column(results_pred)

In [183]:
results_pred %>% head

pid,scenario,kernel,lml,error,is_max
a001,FB Friends,l*p,-87.34479,5.581463,FALSE
a001,FB Friends,l*p+r,-288.72310,8.250953,FALSE
a001,FB Friends,l*r,-117.03230,115.324607,FALSE
a001,FB Friends,l*r*p,-162.17808,125.606908,FALSE
a001,FB Friends,l*r+p,-43.56297,7.039162,TRUE
a001,FB Friends,l+p,-298.99165,3.763067,FALSE


# Third grid plot. 
Identical to the posterior grid plot, but the right column is a scatterplot of prior likelihood vs posterior performance

In [230]:
# Raw LML in the fitting of the GPs onto the prior stage data
scatterplot_data_prior <- lmls_prior_ff %>%
                            select(pid, scenario, kernel, lml)

# Raw Error in the posterior predictions
scatterplot_data_posterior <- results_pred %>%
                                mutate(kernel = readable_kernel(kernel)) %>%
                                select(pid, scenario, kernel, lml, error_posterior=error)

scatterplot_data <- merge(
                            x = scatterplot_data_prior,
                            y = scatterplot_data_posterior,
                            by = c('pid', 'scenario', 'kernel'),
                            suffixes = c('_prior', '_posterior') )

#### Charley Wu Magic

In [231]:
library(MASS)

plot_scenario_scatterplot_wu <- function(data_to_plot, scenario_name, y_max) {
    
    y_max = 11
    
    # Filter and normalize
    aux <- data_to_plot %>% 
            filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
            filter(scenario == scenario_name) %>%
            group_by( pid, scenario ) %>%
            mutate(lml_normalized = standardize_range_one(lml_prior),
                   error_normalized = standardize_range_one(error_posterior), 
                   error_ranking = rank(error_posterior))

    # Only analysis dataframe
    cor1D <- data.frame(lambda1 = aux$lml_normalized, 
                        lambda2 = aux$error_ranking)
    
    # Correlation test
    lambdaCor <- cor.test(cor1D$lambda1, cor1D$lambda2)

    # Bivariate point density
    cor1D$lambdadensity <- fields::interp.surface(MASS::kde2d(cor1D$lambda1, cor1D$lambda2), cor1D[,c("lambda1", "lambda2")])
    
    # Plot
    ggplot(cor1D, aes(lambda1, lambda2, fill=lambdadensity, alpha = standardize_range_one(1/lambdadensity))) +
        geom_point(pch=21, size = 2.0, color='white')+
        geom_smooth(method='lm', color='black', se=FALSE, linetype=2, fullrange=TRUE)+
        scale_fill_gradient(low = "#0091ff", high = "#f0650e") +
        scale_alpha(range = c(.4, 1)) + guides(alpha="none", fill="none") +
        #expand_limits(x = 0, y = 0) +
        #annotate("text", x = 1, y = 6, label = "r == 0.62 * ~~'p<.001'", parse=TRUE, size=6) +
    
        coord_cartesian(ylim = c(0, y_max))  +
        scale_x_continuous(breaks = seq(0, 1, length.out=3)) +
        scale_y_continuous(breaks = seq(1, y_max, length.out=2)) +
        
        xlab('Likelihood in Prior') + 
        ylab('Error in Posterior') +
    
        ggthemes::theme_few() +
        theme(text = element_text(size=10, family="serif"),
                     legend.position = "none")
}

scatter_temp_wu <- plot_scenario_scatterplot_wu(scatterplot_data, 'Temperature', y_max = 30)
scatter_rain_wu <- plot_scenario_scatterplot_wu(scatterplot_data, 'Rain',        y_max = 30)
scatter_sale_wu <- plot_scenario_scatterplot_wu(scatterplot_data, 'Sales',       y_max = 10)
scatter_gymm_wu <- plot_scenario_scatterplot_wu(scatterplot_data, 'Gym members', y_max = 30)
scatter_slry_wu <- plot_scenario_scatterplot_wu(scatterplot_data, 'Salary',      y_max = 30)
scatter_fbfr_wu <- plot_scenario_scatterplot_wu(scatterplot_data, 'FB Friends',  y_max = 30)


#scatter_temp_wu


Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select



In [232]:
if(TRUE){
    pdf("Images/paper_images/grid_plot_posterior_wu_rank_abs.pdf", width=6, height=10)
    multiplot(  posterior_curves_temp,
                posterior_curves_rain,
                posterior_curves_sale,
                posterior_curves_gymm,
                posterior_curves_slry,
                posterior_curves_fbfr,
              
                scatter_temp_wu,
                scatter_rain_wu,
                scatter_sale_wu,
                scatter_gymm_wu,
                scatter_slry_wu,
                scatter_fbfr_wu,
                
                cols = 2)
    dev.off()   
}

detach(package:MASS)

png 
  2

## Statistical tests for the Posterior condition

In [59]:
# Percentage of removed GPs in the Prior condition:
((1 -
lmls_posterior_ff %>%
    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
    nrow / 
read_csv("output/minus-mean-treatment/results_posterior_lmls.csv") %>%
    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
    nrow) * 100) %>%
round(2)

Parsed with column specification:
cols(
  id = col_integer(),
  kernel = col_character(),
  lml = col_double(),
  white_added = col_character(),
  second_exception = col_character()
)


[1] 1.66

##### Linear model in error ranking

In [60]:
to_test_error <- scatterplot_data %>% 
            filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
            group_by( pid, scenario ) %>%
            mutate(lml_prior_standardized = standardize_range_one(lml_prior), # x
                   error_ranking = rank(error_posterior))                     # y

In [61]:
lm_by_scenario <- lapply( split(to_test_error, to_test_error$scenario), 
                          function(data) lm(error_ranking ~ lml_prior_standardized, data))
                              
lm_overall <- lm(error_ranking ~ lml_prior_standardized, data = to_test_error)

#### Eric's cool correlation

In [62]:
to_test_error <- scatterplot_data %>% 
            filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
            group_by( pid, scenario ) %>%
            mutate(lml_prior_standardized = standardize_range_one(lml_prior))

In [63]:
print('Per scenario')
dcor1 <- ddply(to_test_error, ~scenario+pid, summarize, r = cor(x = lml_prior_standardized, rank(error_posterior)))

dout <- ddply(dcor1, ~scenario, summarize, m=cohensD(r))
test_results_per_scenario <- tapply(dcor1$r, dcor1$scenario, t.test)

print('Overall')
dcor2 <- ddply(to_test_error, ~pid, summarize, r = cor(x = lml_prior_standardized, rank(error_posterior)))

t.test(dcor2$r)
cohensD(dcor2$r) %>% round(2)

[1] "Per scenario"
[1] "Overall"



	One Sample t-test

data:  dcor2$r
t = -9.3949, df = 118, p-value = 5.429e-16
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.15066086 -0.09820455
sample estimates:
 mean of x 
-0.1244327 


[1] 0.86

In [64]:
paste0(  print_test_results('Temperature', 'temperature'),
        print_test_results('Rain', 'rain'),
        print_test_results('Sales', 'sales'),
        print_test_results('Gym members', 'gym membership'),
        print_test_results('Salary', 'salary'),
        print_test_results('FB Friends', 'Facebook friends')) %>% print

[1] ", $\rho=-0.23$ ($t(118)=-6.77, p<0.001, d=0.62$) for the temperature data, $\rho=-0.25$ ($t(118)=-8.54, p<0.001, d=0.78$) for the rain data, $\rho=-0.1$ ($t(118)=-3.37, p0.00102089937190916, d=0.31$) for the sales data, $\rho=-0.16$ ($t(118)=-4.6, p<0.001, d=0.42$) for the gym membership data, $\rho=-0.01$ ($t(118)=-0.24, p0.814323148477005, d=0.02$) for the salary data, $\rho=-0.14$ ($t(118)=-4.35, p<0.001, d=0.4$) for the Facebook friends data"


# Trend damping
### Goal: Proportion of damped predictions. Participants vs GP Predictions.

###### Calculate underlying trend

In [181]:
df <- dat %>%
        group_by(condition, scenario, day) %>%
        summarize

df$slope_scale <- mapvalues(df$condition,
                              from = c("Prior", "Posterior-Positive", "Posterior-Stable", "Posterior-Negative"),
                              to   = c(0, 1, 0, -1))

df$range <- mapvalues(df$scenario,
                              from = c("Temperature", "Rain", "Sales", "Gym members", "Salary", "FB Friends"),
                              to   = c( 40--10,        100-0,  5000-0,  50-0,          50-0,     1000-0))

df$y_intercept <- mapvalues(df$scenario,
                              from = c("Temperature", "Rain", "Sales", "Gym members", "Salary", "FB Friends"),
                              to   = c( 15,            30,     2500,    25,            20,       500))


# Transformations to numbers
df$slope_scale <- as.numeric(levels(df$slope_scale))[df$slope_scale]
df$range <- as.numeric(levels(df$range))[df$range]
df$y_intercept <- as.numeric(levels(df$y_intercept))[df$y_intercept]

# Calculations
df$last_point <- df$y_intercept + 0.05 * df$range * df$slope_scale * 4 

df$slope <- (df$last_point - df$y_intercept) / ((365-31)-1)

df$underlying_trend <- ((df$day-1) * df$slope + df$y_intercept)

# Subtract the mean of each underlying trend
#df <- df %>% 
#        group_by(condition, scenario) %>%
#        mutate(underlying_trend = underlying_trend - mean(underlying_trend))

In [184]:
# Watching the columns
all_pred_data %>% head(3) # This has the day to day data
df %>% head(3)            # This has the underlying trend
results_pred %>% head(3)  # This has the error (against prediction)

all_pred_data %>% nrow
df %>% nrow

pid,scenario,day,cid,kernel,value,lml,value_participant,condition,noise
a001,FB Friends,1001,6,l+r+p,156.2030,-334.52795,255.7962,Posterior-Negative,3
a001,FB Friends,1001,6,l+r,112.2065,-72.20523,255.7962,Posterior-Negative,3
a001,FB Friends,1001,6,p*r,382.4196,-323.93799,255.7962,Posterior-Negative,3


condition,scenario,day,slope_scale,range,y_intercept,last_point,slope,underlying_trend
Prior,Temperature,31,0,50,15,15,0,15
Prior,Temperature,32,0,50,15,15,0,15
Prior,Temperature,33,0,50,15,15,0,15


pid,scenario,kernel,lml,error,is_max
a001,FB Friends,l*p,-87.34479,5.581463,FALSE
a001,FB Friends,l*p+r,-288.72310,8.250953,FALSE
a001,FB Friends,l*r,-117.03230,115.324607,FALSE


[1] 1691556

[1] 33576

In [200]:
to_merge_1 <- all_pred_data %>% 
                    select(cid, pid, condition, scenario, kernel, day, value_model = value, value_participant)

to_merge_2 <- results_pred %>% 
                    select(pid, scenario, kernel, lml_prior = lml, error_posterior=error, is_max_posterior=is_max)

first_merge <- merge( x = to_merge_1,
                      y = to_merge_2,
                      by = c('pid', 'scenario', 'kernel'),
                      all.y=TRUE)

data_with_trend <- merge( x = first_merge,
                          y = df %>% select(condition, scenario, day, underlying_trend),
                          by = c('condition', 'scenario', 'day'),
                          all.x = TRUE)

In [225]:
# Filter out the other scenarios. Leave only SALES/GYM MEMBERS
data_with_trend_f <- data_with_trend %>%
                                filter(condition == 'Posterior-Positive' | condition == 'Posterior-Negative') %>%
                                #filter(scenario == 'Sales') %>%
                                filter(scenario == 'Gym members') %>%
                                filter(is_max_posterior)

In [226]:
# Which values are "damped"?
add_damped_column <- function(dataset, condition, column_name, new_column_name) {
    
    if(condition == 'Posterior-Negative') {
        damped_column <- dataset[dataset$condition==condition, column_name] > dataset[dataset$condition==condition, 'underlying_trend'] 
    }else if(condition == 'Posterior-Positive') {
        damped_column <- dataset[dataset$condition==condition, column_name] < dataset[dataset$condition==condition, 'underlying_trend'] 
    }
    
    dataset[dataset$condition == condition, new_column_name] <- damped_column
    
    return(dataset)
}
 
# Calculate the 'damped' boolean
data_with_trend_ff <- add_damped_column(data_with_trend_f, 'Posterior-Negative', 'value_participant', new_column_name='damped_participant')
data_with_trend_ff <- add_damped_column(data_with_trend_ff, 'Posterior-Positive', 'value_participant', new_column_name='damped_participant')
    
data_with_trend_ff <- add_damped_column(data_with_trend_ff, 'Posterior-Negative', 'value_model', new_column_name='damped_model')
data_with_trend_ff <- add_damped_column(data_with_trend_ff, 'Posterior-Positive', 'value_model', new_column_name='damped_model')

In [227]:
n = 1e3

damped_proportion_participants <- data_with_trend_ff %>%
                                        group_by(condition, day) %>%
                                        summarize(damped_proportion = bp2(damped_participant, n=n)[[1]],
                                                  lo_ci = bp2(damped_participant, n=n)[[2]],
                                                  hi_ci = bp2(damped_participant, n=n)[[3]])

damped_proportion_gps <- data_with_trend_ff %>%
                                        group_by(condition, day) %>%
                                        summarize(damped_proportion = bp2(damped_model, n=n)[[1]],
                                                  lo_ci = bp2(damped_model, n=n)[[2]],
                                                  hi_ci = bp2(damped_model, n=n)[[3]])

rm(n)

# Merge the data into one dataframe
damped_proportion_participants$type <- 'Participants'
damped_proportion_gps$type <- 'Models'

damped_proportion <- rbind(damped_proportion_participants, damped_proportion_gps)

damped_proportion$type <- factor(damped_proportion$type, levels=c('Participants', 'Models'))

In [228]:
damped_plot <- damped_proportion %>%
    ggplot(aes(x=day, y=damped_proportion, group=condition, colour=condition)) +
            geom_line(size=1) +
            geom_ribbon(aes(ymin=lo_ci, ymax=hi_ci), alpha=0.2) +
            ylab("Damped proportion in XX scenario") +
            scale_x_continuous( lim = c(min(damped_proportion$day), 365*3), 
                                breaks=c(0, 365, 365*2, 365*3), 
                                labels=c('Y1', 'Y2', 'Y3', 'Y4') ) +
            theme_bw() + 
            #ggthemes::theme_few() + 
            theme( text = element_text(size=12, family="serif"),
                   legend.position = "bottom",
                   axis.title.x=element_blank(),
                   legend.title = element_blank() ) +
            ggthemes::scale_color_solarized() +
            facet_grid(type ~ .)

if(TRUE){ #save_damped_proportion_pdf
    pdf("Images/paper_images/damped_proportion_2_gym.pdf", width=6, height=4)
    print(damped_plot)
    dev.off()
}

Warning message:
"Removed 134 rows containing missing values (geom_path)."

png 
  2

In [175]:
# Merging the data with the underlying trend
data_with_trend <- merge( 
    x = all_pred_data,

    y = df %>% select(condition, scenario, day, underlying_trend),

    by = c('condition', 'scenario', 'day')
)

data_with_trend <- cbind(row_id = rownames(data_with_trend), data_with_trend)

data_with_trend <- merge(   # Original dataset
                            x = data_with_trend,

                            # Mark the best-fitting kernel in each case
                            y = data_with_trend %>%
                                    filter(nchar(kernel)!= 1) %>%
                                    group_by(pid, scenario) %>%
                                    mutate(is_max = lml == max(lml)) %>%
                                    ungroup %>%
                                    filter(is_max) %>%
                                    select(row_id, is_max),

                            by = 'row_id',
                            all.x = TRUE
                )

data_with_trend$is_max <- !(is.na(data_with_trend$is_max))

# Filter out the unnecessary conditions
data_with_trend <- data_with_trend %>%
                                filter(condition == 'Posterior-Positive' |
                                       condition == 'Posterior-Negative')

# Filter out the other scenarios. Leave only SALES/GYM MEMBERS
data_with_trend <- data_with_trend %>%
                                filter(scenario == 'Sales')
                                #filter(scenario == 'Gym members')

# Only have the best-fitting kernel(of the Posterior condition)
data_with_trend <- data_with_trend %>%
                                filter(is_max)

In [176]:
# Should be zero because every curve and day combination should only have 1 datapoint
print('should be zero:')

data_with_trend %>% 
    group_by(cid, day) %>%
    filter(length(day) != 1) %>% nrow

data_with_trend %>% 
    group_by(cid, day, kernel) %>%
    filter(length(kernel) != 1) %>% nrow

[1] "should be zero:"


[1] 0

[1] 0

In [177]:
# Which values are "damped"?
add_damped_column <- function(dataset, condition, column_name, new_column_name) {
    
    if(condition == 'Posterior-Negative') {
        damped_column <- dataset[dataset$condition==condition, column_name] > dataset[dataset$condition==condition, 'underlying_trend'] 
    }else if(condition == 'Posterior-Positive') {
        damped_column <- dataset[dataset$condition==condition, column_name] < dataset[dataset$condition==condition, 'underlying_trend'] 
    }
    
    dataset[dataset$condition == condition, new_column_name] <- damped_column
    
    return(dataset)
}
 
# Calculate the 'damped' boolean
data_with_trend <- add_damped_column(data_with_trend, 'Posterior-Negative', 'value_participant', new_column_name='damped_participant')
data_with_trend <- add_damped_column(data_with_trend, 'Posterior-Positive', 'value_participant', new_column_name='damped_participant')
    
data_with_trend <- add_damped_column(data_with_trend, 'Posterior-Negative', 'value', new_column_name='damped_gp')
data_with_trend <- add_damped_column(data_with_trend, 'Posterior-Positive', 'value', new_column_name='damped_gp')

##### Calculate the damped proportion, per condition~day

In [178]:
n = 1e3

damped_proportion_participants <- data_with_trend %>%
                                        group_by(condition, day) %>%
                                        summarize(damped_proportion = bp2(damped_participant, n=n)[[1]],
                                                  lo_ci = bp2(damped_participant, n=n)[[2]],
                                                  hi_ci = bp2(damped_participant, n=n)[[3]])

damped_proportion_gps <- data_with_trend %>%
                                        group_by(condition, day) %>%
                                        summarize(damped_proportion = bp2(damped_gp, n=n)[[1]],
                                                  lo_ci = bp2(damped_gp, n=n)[[2]],
                                                  hi_ci = bp2(damped_gp, n=n)[[3]])

rm(n)

# Merge the data into one dataframe
damped_proportion_participants$type <- 'Participants'
damped_proportion_gps$type <- 'Models'

damped_proportion <- rbind(damped_proportion_participants, damped_proportion_gps)

damped_proportion$type <- factor(damped_proportion$type, levels=c('Participants', 'Models'))

In [179]:
damped_plot <- damped_proportion %>%
    ggplot(aes(x=day, y=damped_proportion, group=condition, colour=condition)) +
            geom_line(size=1) +
            geom_ribbon(aes(ymin=lo_ci, ymax=hi_ci), alpha=0.2) +
            ylab("Damped proportion in XX scenario") +
            scale_x_continuous( lim = c(min(damped_proportion$day), 365*3), 
                                breaks=c(0, 365, 365*2, 365*3), 
                                labels=c('Y1', 'Y2', 'Y3', 'Y4') ) +
            theme_bw() + 
            #ggthemes::theme_few() + 
            theme( text = element_text(size=12, family="serif"),
                   legend.position = "bottom",
                   axis.title.x=element_blank(),
                   legend.title = element_blank() ) +
            ggthemes::scale_color_solarized() +
            facet_grid(type ~ .)

if(TRUE){ #save_damped_proportion_pdf
    pdf("Images/paper_images/damped_proportion.pdf", width=6, height=4)
    print(damped_plot)
    dev.off()
}

Warning message:
"Removed 134 rows containing missing values (geom_path)."

png 
  2

### Trend damping statistical tests
###### (i) Overall proportion of damped predictions

In [80]:
damped_to_test <- merge(
        x = damped_proportion %>% filter(type == 'Participants') %>% select(condition, day, damped_proportion),

        y = damped_proportion %>% filter(type == 'Models') %>% select(condition, day, damped_proportion),

        by = c('condition', 'day'),

        suffixes = c('_participants', '_models')) %>%

        mutate(subtraction = damped_proportion_participants - damped_proportion_models) %>%

        filter(day <= 365 * 2)

In [81]:
c = 'Posterior-Positive'
print(c)
fit <- lm( damped_proportion_participants ~ damped_proportion_models, 
            data = damped_to_test %>% filter(condition == c))
summary(fit)


c = 'Posterior-Negative'
print(c)
fit <- lm( damped_proportion_participants ~ damped_proportion_models, 
            data = damped_to_test %>% filter(condition == c))
summary(fit)

[1] "Posterior-Positive"



Call:
lm(formula = damped_proportion_participants ~ damped_proportion_models, 
    data = damped_to_test %>% filter(condition == c))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.07634 -0.00871 -0.00848  0.02129  0.06412 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)              -0.42589    0.06137  -6.939 1.08e-09 ***
damped_proportion_models  1.40460    0.06377  22.026  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.02671 on 77 degrees of freedom
Multiple R-squared:  0.863,	Adjusted R-squared:  0.8612 
F-statistic: 485.1 on 1 and 77 DF,  p-value: < 2.2e-16


[1] "Posterior-Negative"



Call:
lm(formula = damped_proportion_participants ~ damped_proportion_models, 
    data = damped_to_test %>% filter(condition == c))

Residuals:
      Min        1Q    Median        3Q       Max 
-0.066701 -0.006701  0.009595  0.009595  0.093546 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)              -0.46639    0.09056   -5.15 1.94e-06 ***
damped_proportion_models  1.45679    0.09250   15.75  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.02227 on 77 degrees of freedom
Multiple R-squared:  0.7631,	Adjusted R-squared:   0.76 
F-statistic:   248 on 1 and 77 DF,  p-value: < 2.2e-16


###### (ii) First damped point

In [82]:
first_point_damped <- merge(
    x = data_with_trend %>% 
            group_by(pid, condition, scenario) %>%
            filter(damped_participant) %>%
            summarize(min_day = min(day)),
    
    y = data_with_trend %>% 
            group_by(pid, condition, scenario) %>%
            filter(damped_gp) %>%
            summarize(min_day = min(day)),
    
    by = c('pid', 'condition', 'scenario'),

    suffixes = c('_participants', '_models')) %>%

    mutate(subtraction = min_day_participants - min_day_models)# %>% filter(min_day_participants != 336)

In [83]:
c <- 'Posterior-Positive'
print(c)
fit <- lm(min_day_participants ~ min_day_models, data = first_point_damped %>% filter(condition == c))
summary(fit)

c <- 'Posterior-Negative'
print(c)
fit <- lm(min_day_participants ~ min_day_models, data = first_point_damped %>% filter(condition == c))
summary(fit)

[1] "Posterior-Positive"



Call:
lm(formula = min_day_participants ~ min_day_models, data = first_point_damped %>% 
    filter(condition == c))

Residuals:
   Min     1Q Median     3Q    Max 
-64.96 -19.86 -19.86 -19.86 310.14 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)  
(Intercept)    135.1419   131.3404   1.029   0.3104  
min_day_models   0.6569     0.3806   1.726   0.0929 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 64.78 on 36 degrees of freedom
Multiple R-squared:  0.07644,	Adjusted R-squared:  0.05079 
F-statistic:  2.98 on 1 and 36 DF,  p-value: 0.09289


[1] "Posterior-Negative"



Call:
lm(formula = min_day_participants ~ min_day_models, data = first_point_damped %>% 
    filter(condition == c))

Residuals:
    Min      1Q  Median      3Q     Max 
-24.952  -7.146  -7.146  -7.146 149.401 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -444.8822   110.4283  -4.029  0.00031 ***
min_day_models    2.3453     0.3247   7.223 2.78e-08 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 30.66 on 33 degrees of freedom
Multiple R-squared:  0.6125,	Adjusted R-squared:  0.6008 
F-statistic: 52.17 on 1 and 33 DF,  p-value: 2.779e-08


In [85]:
caux = 'Posterior-Positive' 
print(caux)
first_point_damped %>% 
    filter(condition == caux) %>%
    select(subtraction) %>%
    t.test

caux = 'Posterior-Negative' 
print(caux)
first_point_damped %>% 
    filter(condition == caux) %>%
    select(subtraction) %>%
    t.test

[1] "Posterior-Positive"



	One Sample t-test

data:  .
t = 1.6319, df = 37, p-value = 0.1112
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -4.133361 38.343888
sample estimates:
mean of x 
 17.10526 


[1] "Posterior-Negative"



	One Sample t-test

data:  .
t = 1.9288, df = 34, p-value = 0.06214
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.6512242 24.9369385
sample estimates:
mean of x 
 12.14286 


###### (iii) Damped proportion per participant

In [86]:
up_bound_day <- 365*2

damped_mean_per_pid <- merge(
    x = data_with_trend %>% 
            group_by(pid, condition, scenario) %>%
            filter(day < up_bound_day) %>%
            summarize(damped_mean = mean(damped_participant)),
    
    y = data_with_trend %>% 
            group_by(pid, condition, scenario) %>%
            filter(day < up_bound_day) %>%
            summarize(damped_mean = mean(damped_gp)),
    
    by = c('pid', 'condition', 'scenario'),

    suffixes = c('_participants', '_models')) %>%

    mutate(subtraction = damped_mean_participants - damped_mean_models)# %>% filter(min_day_participants != 336)

In [87]:
damped_mean_per_pid %>% 
    filter(damped_mean_participants != 1) %>%
    select(subtraction) %>%
    t.test


	One Sample t-test

data:  .
t = -3.4548, df = 12, p-value = 0.004763
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.38583425 -0.08738873
sample estimates:
 mean of x 
-0.2366115 


Plots of the raw data 

In [88]:
pos_trend <- data_with_trend %>% 
                filter(condition == 'Posterior-Positive') %>%
                select(day, underlying_trend) %>%
                arrange(day) %>%
                unique

neg_trend <- data_with_trend %>% 
                filter(condition == 'Posterior-Negative') %>%
                select(day, underlying_trend) %>%
                arrange(day) %>%
                unique

In [89]:
pdf("Images/paper_images/gp_prediction_raw_data.pdf", width=13, height=6)

if(FALSE){
    data_with_trend %>% 
    ggplot() +
    geom_line(aes(x=day, y=value, group=pid, color=condition)) +
    geom_line(data=pos_trend, aes(x=day, y=underlying_trend, group=1, alpha=0.5), size=2) +
    geom_line(data=neg_trend, aes(x=day, y=underlying_trend, group=2, alpha=0.5), size=2) +
    xlim(c(330, 850)) +
    ylim(c(-1000, 10000)) +
    ggthemes::theme_few() 
    #theme(legend.position="none")

    dev.off()
}

# Posterior analysis, proportion of match

In [90]:
error_posterior_incomplete <- results_pred %>% 
                                    mutate(kernel = readable_kernel(kernel),
                                           true_lml = lml,
                                           lml = -error) # This is just for function reasons

error_posterior <- merge( x = error_posterior_incomplete,
                          y = lmls_posterior_ff %>% select(pid, condition, scenario) %>% unique,
                          by = c('pid', 'scenario'))

print('Should be TRUE:')
print(error_posterior %>% nrow == 7724)

[1] "Should be TRUE:"
[1] TRUE


In [93]:
lmls_prior_ff %>% head
error_posterior %>% head

ks,cid,kernel,lml,white_added,second_exception,pid,scenario,lo_bound,up_bound
l+p+rFB Friends,552,l+p+r,-1332.3942,False,False,a092,FB Friends,-4051.607,3517.197
l+p+rFB Friends,516,l+p+r,582.2864,False,False,a086,FB Friends,-4051.607,3517.197
l+p+rFB Friends,120,l+p+r,-354.7262,False,False,a020,FB Friends,-4051.607,3517.197
l+p+rFB Friends,690,l+p+r,615.8284,False,False,a116,FB Friends,-4051.607,3517.197
l+p+rFB Friends,60,l+p+r,-1388.2844,False,False,a010,FB Friends,-4051.607,3517.197
l+p+rFB Friends,354,l+p+r,-1265.4150,True,False,a059,FB Friends,-4051.607,3517.197


pid,scenario,kernel,lml,error,is_max,true_lml,condition
a001,FB Friends,l×p,-5.581463,5.581463,FALSE,-87.34479,Posterior-Negative
a001,FB Friends,l×p+r,-8.250953,8.250953,FALSE,-288.72310,Posterior-Negative
a001,FB Friends,l×r,-115.324607,115.324607,FALSE,-117.03230,Posterior-Negative
a001,FB Friends,l×p×r,-125.606908,125.606908,FALSE,-162.17808,Posterior-Negative
a001,FB Friends,l×r+p,-7.039162,7.039162,TRUE,-43.56297,Posterior-Negative
a001,FB Friends,l+p,-3.763067,3.763067,FALSE,-298.99165,Posterior-Negative


In [95]:
# Remove the non-compositional kernels AND leave only the best-fitting kernel for each (pid, scenario)
best_fitting_per_pid_scenario <- function(lmls_ff) {
   lmls_ff %>%
    filter( kernel != 'l', kernel != 'p', kernel != 'r' ) %>%
    group_by( pid, condition, scenario ) %>%
    summarize( best_kernel = kernel[which.max(lml)],
               lml         =    lml[which.max(lml)])  
}

best_fit_and_merge <- function(prior_data, posterior_data){
    # Adding condition to the prior_data
    prior_data$condition <- 'Prior'
    
    best_in_prior <- best_fitting_per_pid_scenario(prior_data)
    best_in_posterior <- best_fitting_per_pid_scenario(posterior_data)

    # Merging the best fitting compositions
    aux_best_fitting_kernels <- merge(
                                x = best_in_prior,
                                y = best_in_posterior,
                                by = c('pid', 'scenario'),
                                suffixes = c('_prior', '_posterior')
    )
    
    return(aux_best_fitting_kernels)
}

best_fitting_kernels <- best_fit_and_merge(lmls_prior_ff, error_posterior)

In [96]:
# Now we have to check and calculate the proportions of match
match_per_condition <- best_fitting_kernels %>%
    mutate(match = (best_kernel_prior == best_kernel_posterior)) %>%
    group_by(condition_posterior) %>%
    summarize(match_proportion = bp2(match, alpha=0.05, round_n=3)[1], 
              lo_ci            = bp2(match, alpha=0.05, round_n=3)[2], 
              hi_ci            = bp2(match, alpha=0.05, round_n=3)[3])

match_per_scenario <- best_fitting_kernels %>%
    mutate(match = (best_kernel_prior == best_kernel_posterior)) %>%
    group_by(scenario) %>%
    summarize(match_proportion = bp2(match, alpha=0.05, round_n=3)[1], 
              lo_ci            = bp2(match, alpha=0.05, round_n=3)[2], 
              hi_ci            = bp2(match, alpha=0.05, round_n=3)[3])

Observing the results:

In [101]:
print(paste('Reference:', round(1/11, 3)))

match_per_condition
match_per_scenario

[1] "Reference: 0.091"


condition_posterior,match_proportion,lo_ci,hi_ci
Posterior-Negative,0.139,0.097,0.185
Posterior-Positive,0.130,0.088,0.176
Posterior-Stable,0.126,0.084,0.172


scenario,match_proportion,lo_ci,hi_ci
FB Friends,0.076,0.034,0.126
Gym members,0.168,0.101,0.235
Rain,0.185,0.118,0.261
Salary,0.127,0.067,0.185
Sales,0.092,0.042,0.151
Temperature,0.143,0.084,0.210


#### Actual tests

##### Per condition (not used)

In [102]:
x_n_per_condition <- best_fitting_kernels %>%
                        mutate(match = (best_kernel_prior == best_kernel_posterior)) %>%
                        group_by(condition_posterior) %>%
                        summarize(x = sum(match), n = length(match))

x_n_per_condition

condition_posterior,x,n
Posterior-Negative,33,238
Posterior-Positive,31,238
Posterior-Stable,30,238


In [103]:
binom_test_special <- function (dataset, condition_name) {
    x <- (dataset %>% filter(condition_posterior == condition_name))$x
    n <- (dataset %>% filter(condition_posterior == condition_name))$n
    
    print(condition_name)
    binom.test(x, n, p=1/11, alternative="two.sided", conf.level=0.95)
}

In [104]:
binom_test_special(x_n_per_condition, "Posterior-Negative")
binom_test_special(x_n_per_condition, "Posterior-Stable")
binom_test_special(x_n_per_condition, "Posterior-Positive")

[1] "Posterior-Negative"



	Exact binomial test

data:  x and n
number of successes = 33, number of trials = 238, p-value = 0.01707
alternative hypothesis: true probability of success is not equal to 0.09090909
95 percent confidence interval:
 0.09740411 0.18917909
sample estimates:
probability of success 
             0.1386555 


[1] "Posterior-Stable"



	Exact binomial test

data:  x and n
number of successes = 30, number of trials = 238, p-value = 0.06999
alternative hypothesis: true probability of success is not equal to 0.09090909
95 percent confidence interval:
 0.08668616 0.17503235
sample estimates:
probability of success 
             0.1260504 


[1] "Posterior-Positive"



	Exact binomial test

data:  x and n
number of successes = 31, number of trials = 238, p-value = 0.04165
alternative hypothesis: true probability of success is not equal to 0.09090909
95 percent confidence interval:
 0.09024432 0.17976189
sample estimates:
probability of success 
             0.1302521 


##### By scenario

In [105]:
x_n_per_scenario <- best_fitting_kernels %>%
                        mutate(match = (best_kernel_prior == best_kernel_posterior)) %>%
                        group_by(scenario) %>%
                        summarize(x = sum(match), n = length(match))

x_n_per_scenario

scenario,x,n
FB Friends,9,119
Gym members,20,119
Rain,22,119
Salary,15,119
Sales,11,119
Temperature,17,119


In [122]:
binom_test_special_scenario <- function (dataset, scenario_name) {
    x <- (dataset %>% filter(scenario == scenario_name))$x
    n <- (dataset %>% filter(scenario == scenario_name))$n
    
    print(scenario_name)
    binom.test(x, n, p=1/11, alternative="two.sided", conf.level=0.95)
}

for(scn in readable_scenarios){
    print(binom_test_special_scenario(x_n_per_scenario, scn))
}

[1] "Temperature"

	Exact binomial test

data:  x and n
number of successes = 43, number of trials = 119, p-value = 7.008e-16
alternative hypothesis: true probability of success is not equal to 0.09090909
95 percent confidence interval:
 0.2753057 0.4544629
sample estimates:
probability of success 
             0.3613445 

[1] "Rain"

	Exact binomial test

data:  x and n
number of successes = 63, number of trials = 119, p-value < 2.2e-16
alternative hypothesis: true probability of success is not equal to 0.09090909
95 percent confidence interval:
 0.4357869 0.6215381
sample estimates:
probability of success 
             0.5294118 

[1] "Sales"

	Exact binomial test

data:  x and n
number of successes = 17, number of trials = 119, p-value = 0.0551
alternative hypothesis: true probability of success is not equal to 0.09090909
95 percent confidence interval:
 0.0854768 0.2188484
sample estimates:
probability of success 
             0.1428571 

[1] "Gym members"

	Exact binomial test

da

# Posterior analysis, participants' data, proportion of match

In [107]:
# Prior data
lmls_prior_ff$condition <- 'Prior'

# Posterior data (participants (i.e., no evidence))
lmls_posterior_participants_ff <- get_filtered_data( "output/minus-mean-treatment/results_posterior_participants_lmls.csv",
                                                     "data/for_composititional_analysis_posterior.csv" )

# Readable kernel composition
lmls_posterior_participants_ff$kernel <- readable_kernel(lmls_posterior_participants_ff$kernel)

Parsed with column specification:
cols(
  id = col_integer(),
  kernel = col_character(),
  lml = col_double(),
  white_added = col_character(),
  second_exception = col_character()
)
Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_integer(),
  id = col_integer(),
  pid = col_character(),
  scenario = col_character(),
  x = col_integer(),
  y = col_double(),
  condition = col_character()
)


In [113]:
best_fitting_kernels <- best_fit_and_merge(lmls_prior_ff, 
                                           lmls_posterior_participants_ff)

# Now we have to check and calculate the proportions of match
match_per_scenario <- best_fitting_kernels %>%
    mutate(match = (best_kernel_prior == best_kernel_posterior)) %>%
    group_by(scenario) %>%
    summarize(match_proportion = bp2(match, alpha=0.05, round_n=3)[1], 
              lo_ci            = bp2(match, alpha=0.05, round_n=3)[2], 
              hi_ci            = bp2(match, alpha=0.05, round_n=3)[3])

match_per_scenario

scenario,match_proportion,lo_ci,hi_ci
FB Friends,0.311,0.227,0.395
Gym members,0.353,0.269,0.437
Rain,0.529,0.445,0.622
Salary,0.336,0.252,0.420
Sales,0.142,0.084,0.210
Temperature,0.361,0.277,0.445


##### Statistical tests per scenario:

In [147]:
x_n_per_scenario <- best_fitting_kernels %>%
                        mutate(match = (best_kernel_prior == best_kernel_posterior)) %>%
                        group_by(scenario) %>%
                        summarize(x = sum(match), n = length(match))

x_n_per_scenario

for(scn in readable_scenarios){
    #print(binom_test_special_scenario(x_n_per_scenario, scn))
}

# Readable:
to_report = ''

for(scn in readable_scenarios){
    
    test_results_per_scenario <- binom_test_special_scenario(x_n_per_scenario, scn)
    
    proportion <- test_results_per_scenario[[5]][[1]] %>% round(2)
    
    df <- test_results_per_scenario[[2]]
    p_val <- ifelse(test_results_per_scenario[[3]] < 0.001, 
                "<0.001", 
                test_results_per_scenario[[3]])
    t <- test_results_per_scenario[[1]] %>% round(2)
    
    t_string <- paste0("$r=", proportion, "$ ($t(", df, ")=", t, "$, $p", p_val, "$)")
    
    to_return_scenario <- paste0(t_string, " for the ", scn, " data, ")
    
    to_report <- paste0(to_report, to_return_scenario)
}

print(to_report)

scenario,x,n
FB Friends,37,119
Gym members,42,119
Rain,63,119
Salary,40,119
Sales,17,119
Temperature,43,119


[1] "Temperature"
[1] "Rain"
[1] "Sales"
[1] "Gym members"
[1] "Salary"
[1] "FB Friends"
[1] "$r=0.36$ ($t(119)=43$, $p<0.001$) for the Temperature data, $r=0.53$ ($t(119)=63$, $p<0.001$) for the Rain data, $r=0.14$ ($t(119)=17$, $p0.0551045262539838$) for the Sales data, $r=0.35$ ($t(119)=42$, $p<0.001$) for the Gym members data, $r=0.34$ ($t(119)=40$, $p<0.001$) for the Salary data, $r=0.31$ ($t(119)=37$, $p<0.001$) for the FB Friends data, "


#### To graph:

In [108]:
# Calculate the best fitting compositional kernels in each condition
best_in_prior <- best_fitting_per_pid_scenario(lmls_prior_ff)
best_in_posterior <- best_fitting_per_pid_scenario(lmls_posterior_participants_ff)

# Merging the best fitting compositions
best_fitting_kernels <- merge(
                            x = best_in_prior,
                            y = best_in_posterior,
                            by = c('pid', 'scenario'),
                            suffixes = c('_prior', '_posterior')
)

In [110]:
# Random
(1 / (length(kernels) - 3)) %>% round(3)

[1] 0.091

In [ ]:
# Now we have to check and calculate the proportions of match
match_per_condition <- best_fitting_kernels %>%
    mutate(match = (best_kernel_prior == best_kernel_posterior)) %>%
    group_by(condition_posterior) %>%
    summarize(match_proportion = bp2(match, alpha=0.01)[1], 
              lo_ci            = bp2(match, alpha=0.01)[2], 
              hi_ci            = bp2(match, alpha=0.01)[3])

match_per_scenario <- best_fitting_kernels %>%
    mutate(match = (best_kernel_prior == best_kernel_posterior)) %>%
    group_by(scenario) %>%
    summarize(match_proportion = bp2(match, alpha=0.01)[1], 
              lo_ci            = bp2(match, alpha=0.01)[2], 
              hi_ci            = bp2(match, alpha=0.01)[3])

##### Plot the matching

In [ ]:
plot_consistence = function(consistence_data, title, xlab, ylim_hi=0.45, ylab="Proportion of Match", x_grouping){
   consistence_data %>%
            ggplot(aes_string(x = x_grouping, y = 'match_proportion')) + 
                geom_bar(stat="identity") +
                scale_fill_manual(values=c("grey35", "black")) +
                coord_cartesian (ylim=c(0, ylim_hi)) +
                labs(title = title) +
                ggthemes::theme_few() +
                theme(legend.position="none") +
                xlab(xlab) + 
                ylab(ylab) +
                geom_errorbar(aes(ymin=as.numeric(lo_ci), ymax=as.numeric(hi_ci)),
                                  width=.3,                   
                                  position=position_dodge(.9)) +
                theme(#axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                      text = element_text(size=12, family="serif"),
                      plot.title = element_text(hjust = 0.5)) +
                geom_hline(yintercept = 1/11, linetype="dashed") #chance value
}

In [ ]:
# VISUALIZATION stuff
posterior_conditions <- c('Posterior-Positive', 'Posterior-Stable', 'Posterior-Negative')
readable_posterior_conditions <- c('Positive', 'Stable', 'Negative')

readable_scenarios_2 <- c("Temperature", "Rain", "Sales", "Gym", "Salary", "FB Friends")

# Change the order and readability of the data
## Conditions
match_per_condition$condition_posterior <- mapvalues( match_per_condition$condition_posterior,
                                                      from = posterior_conditions,
                                                      to   = readable_posterior_conditions )

match_per_condition$condition_posterior <- factor( match_per_condition$condition_posterior,
                                                   levels = readable_posterior_conditions )

## Scenarios
match_per_scenario$scenario <- mapvalues( match_per_scenario$scenario,
                                          from = readable_scenarios,
                                          to   = readable_scenarios_2 )

match_per_scenario$scenario <- factor( match_per_scenario$scenario,
                                       levels = readable_scenarios_2 )

In [ ]:
plot_cond <- match_per_condition %>% plot_consistence(title="(a)", xlab="Posterior conditions", x_grouping="condition_posterior")

plot_scen <- match_per_scenario %>% plot_consistence(title="(b)", xlab="Scenarios", x_grouping="scenario")

if(TRUE){
    pdf("Images/paper_images/match_proportion.pdf", width=4, height=6)
    multiplot(plot_cond, plot_scen, cols=1)
    dev.off()
}

# Debug
###### Prior participant


In [ ]:
raw_prior_data <- read_csv('data/for_composititional_analysis_prior.csv')

In [ ]:
raw_prior_data %>%
    filter(pid == 'a008', scenario == 'Sales') %>%
    ggplot(aes(x, y)) + geom_line()

## Posterior 

In [ ]:
# Sequence of days to analyze:
new_sequence = seq(31, 1426, 5)

In [ ]:
# Inner join. 
# This is the predictions data merged with the best performing (and filtered) kernels in the posterior condition
new_pred <- merge( 
            # Predictions (full-Bayesian):
            x = predictions %>%
                    filter(day %in% new_sequence),

            # Kernels after the filters:
            y = lmls_posterior_ff %>%
                    select(cid=id, pid, scenario, kernel, lml),

            by = c('cid', 'kernel')
)

# Participants' data
new_participants_data <- dat %>% 
                        filter(#day > 365-31, #remove evidence
                               condition != 'Prior', #only Posterior conditions
                               day %in% new_sequence) %>%
                        select(pid = id, day, value_participant=value, scenario, condition, noise)

# Merge the predictions with the real participants' data
new_all_pred_data <- merge(
                # Predictions + Kernel posterior analysis:
                x = new_pred,
    
                # Participant's data:
                y = new_participants_data,
    
                by = c('pid', 'scenario', 'day')
)

In [ ]:
# New error dataset (mean-error per participant per scenario per model)
new_error_per_participant <- new_all_pred_data %>%
                            #filter(day <= 365*2) %>% # To Eric: this is something I though about. What do you think about it?
                            group_by(pid, scenario, kernel, lml) %>% 
                            summarize(error = calculate_mean_error(value, value_participant))


new_error_per_participant <- add_is_max_column(new_error_per_participant)


new_error_per_participant <- new_error_per_participant %>%
                                filter(kernel == 'r' | is_max)

# Remove those [pid, kernel] combinations where the R kernel was not present (because it was previously filtered out)
new_error_per_participant_ff <- new_error_per_participant %>%
                                group_by(pid, scenario) %>%
                                filter(length(kernel) > 1)

In [ ]:
to_plot_post <- merge(
                    x = new_all_pred_data,
                    y = new_error_per_participant_ff %>% select(pid, scenario, kernel) %>% unique
                )

to_plot_post$id <- paste(to_plot_post$cid, to_plot_post$kernel)

In [ ]:
# Temperature: a001, a010, a012

In [ ]:
to_plot_post %>% 
    filter(scenario == 'Temperature',
           pid == 'a001' ) %>%
    ggplot() +
    geom_line(aes(x=day, y=value_participant, group=id), color='blue', size=1, alpha=0.8) +
    geom_line(aes(x=day, y=value, group=kernel, color=kernel), size=1, alpha=0.8) +
    ggthemes::theme_few()